<a href="https://colab.research.google.com/github/WRFitch/fyp/blob/main/src/fyp_model_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing
A notebook for testing an exported model. Ideally, this can be considered a part of a model evaluation pipeline, in which a model can be evaluated in greater depth. 

## Setup

### Notebook Setup 

In [ ]:
!pip uninstall -y fastai
!pip install -U --no-cache-dir fastai

In [30]:
from fastai.vision.all import *
from google.colab import drive
#from scipy import stats
from sklearn.metrics import mean_squared_error

import numpy as np 
import os 
import pandas as pd

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%rm -rf /content/fyp/

In [2]:
# Import fyputil library
%cd /content
!git clone https://github.com/WRFitch/fyp.git
%cd fyp/src/fyputil
import constants as c
import fyp_utils as fyputil
%cd /content

/content
Cloning into 'fyp'...
remote: Enumerating objects: 305, done.
remote: Counting objects: 100% (305/305), done.
remote: Compressing objects: 100% (250/250), done.
remote: Total 817 (delta 198), reused 103 (delta 55), pack-reused 512
Receiving objects: 100% (817/817), 143.45 MiB | 32.85 MiB/s, done.
Resolving deltas: 100% (453/453), done.
/content/fyp/src/fyputil
/content


### Data Setup 

In [3]:
# Add a dummy label script to fool fastai into letting us import the model. 
# We're not going to train the model further, so this is fine. 
def getGhgsAsArr(img_path):
  return np.array()

model = load_learner(f"{c.model_dir}/{c.model_name}.pkl")

In [ ]:
ghg_df = pd.read_csv(c.ghg_csv)
ghg_df = fyputil.normGhgDf(ghg_df)
ghg_df

,Unnamed: 0,system:index,SO2_column_number_density,longitude,latitude,CH4_column_volume_mixing_ratio_dry_air,CO_column_number_density,tropospheric_HCHO_column_number_density,tropospheric_NO2_column_number_density,O3_column_number_density
0,134,0_134,2.672824,-0.795009,51.118631,1.846587,3.273310,4.340454,4.529627,1.435862
1,135,0_135,2.486536,-0.786026,51.118631,1.846299,3.270005,4.446930,4.546964,1.435798
2,136,0_136,2.740308,-0.777043,51.118631,1.845863,3.272315,4.323521,4.499055,1.435488
3,137,0_137,3.104459,-0.768060,51.118631,1.843941,3.271314,4.237725,4.479952,1.435175
4,138,0_138,3.176498,-0.759076,51.118631,1.845095,3.268939,4.456948,4.483478,1.435099
...,...,...,...,...,...,...,...,...,...,...
1881,4147,0_4147,2.313291,0.345851,51.379143,1.853428,3.295407,5.897296,5.975961,1.440403
1882,4148,0_4148,2.328240,0.354835,51.379143,1.858412,3.300101,5.582224,5.942072,1.440447
1883,4149,0_4149,2.456615,0.363818,51.379143,1.854189,3.314009,6.073959,5.988985,1.440476
1884,4150,0_4150,2.675886,0.372801,51.379143,1.854189,3.309340,6.224904,5.922944,1.440376


In [ ]:
err_headers = [c.lon, c.lat] + c.ghg_bands
errors = pd.DataFrame(columns = err_headers)
errors.iloc[0:1]

,longitude,latitude,CO_column_number_density,tropospheric_HCHO_column_number_density,tropospheric_NO2_column_number_density,O3_column_number_density,SO2_column_number_density,CH4_column_volume_mixing_ratio_dry_air


## Testing

### Test model against existing data 

In [ ]:
print(c.ghg_bands)

# TODO replace with fyputil, OR re-implement to keep any failed measurements and
# evaluate what you can out of them. Just because they're incomplete, that 
# doesn't make them worthless 
def getGhgs(img_path, df): 
  coords = fyputil.getCoords(str(img_path))
  ghgs = fyputil.getValAt(coords, df)
  concentrations = ghgs[c.ghg_bands]
  if len(concentrations) == 0 : return None 
  if None in concentrations: return None
  # There has to be a cleaner way to do this. Iterating through and then only getting the first line? really? 
  return [tuple(x) for x in concentrations.to_numpy()][0]

['CO_column_number_density', 'tropospheric_HCHO_column_number_density', 'tropospheric_NO2_column_number_density', 'O3_column_number_density', 'SO2_column_number_density', 'CH4_column_volume_mixing_ratio_dry_air']


In [ ]:
mod = 0 

for filename in os.listdir(c.png_dir):
  file_ghgs = getGhgs(filename, ghg_df)
  if file_ghgs == None: continue
  if mod % 1 == 0:
    print(f"predicting ghg gases at {filename}")
    coords = fyputil.getCoords(filename) 
    prediction = model.predict(f"{c.png_dir}/{filename}")[0]

    diffs = [pred - act for pred, act in zip(prediction, file_ghgs)]
    errors.loc[len(errors)] = list(coords) + diffs

    print(tuple(coords))
    print(prediction)
    print(file_ghgs)
    print(diffs)
    print()
  mod += 1 



In [ ]:
errors 

,longitude,latitude,CO_column_number_density,tropospheric_HCHO_column_number_density,tropospheric_NO2_column_number_density,O3_column_number_density,SO2_column_number_density,CH4_column_volume_mixing_ratio_dry_air
0,-0.786026,51.343210,0.064012,0.096601,-0.166440,-0.072270,-0.908078,-0.007811
1,0.345851,51.343210,-1.171380,0.044821,-1.061186,-1.418497,-1.168851,-1.755932
2,0.336868,51.361176,0.029759,-0.363708,-0.573617,-0.008925,0.766215,-0.056323
3,0.282969,51.370160,0.003683,0.423248,-0.850894,-0.057541,-0.221456,-0.048038
4,0.202121,51.379143,-0.000216,0.645236,-0.946186,-0.063093,0.212429,-0.036178
...,...,...,...,...,...,...,...,...
1900,0.318902,51.262362,0.004716,0.213528,-0.169675,-0.023972,-0.175421,-0.056347
1901,0.354835,51.262362,0.017985,0.233505,-0.209146,-0.034433,-0.283268,0.018875
1902,0.372801,51.262362,0.016011,0.171971,-0.351008,-0.016529,-0.382427,-0.020478
1903,0.363818,51.262362,0.018944,0.729534,-0.359175,-0.023644,-0.175394,-0.028944


In [ ]:
errors.to_csv(f"{c.data_dir}/errors.csv")

### Basic stat testing 
- Data exploration 
- RMSE per GHG
- Extract outliers & view images 

In [5]:
errors = pd.read_csv(f"{c.data_dir}/errors.csv")

In [33]:
model_stats = pd.DataFrame(columns = ["stat"] + c.ghg_bands)

In [54]:
def getRmse(series): 
  return np.sqrt(np.mean(series**2))


In [58]:
# Define aggregate metrics 
means = [errors[ghg].mean() for ghg in c.ghg_bands ]
stdevs = [errors[ghg].std() for ghg in c.ghg_bands ]
rmse = [getRmse(errors[ghg]) for ghg in c.ghg_bands ]

model_stats.loc[1] = ["Mean"] + means
model_stats.loc[2] = ["Standard Deviation"] + stdevs 
model_stats.loc[3] = ["RMSE"] + rmse

In [60]:
model_stats

,stat,CO_column_number_density,tropospheric_HCHO_column_number_density,tropospheric_NO2_column_number_density,O3_column_number_density,SO2_column_number_density,CH4_column_volume_mixing_ratio_dry_air
1,Mean,-0.026073,0.038114,0.059857,-0.089485,-0.079536,-0.039068
2,Standard Deviation,0.390438,0.850026,0.712047,0.220735,0.762539,0.611025
3,RMSE,0.391205,0.850658,0.714373,0.238130,0.766477,0.612113
4,functional RMSE,0.391205,0.850658,0.714373,0.238130,0.766477,0.612113


In [ ]:
# Find outliers 

### Sample images vs predictions 
what regions are easier to predict than others? 

create accuracy heatmap 

### Activation Mapping